In [104]:
import numpy as np
import pandas as pd
import warnings
import math
import os,glob
import matplotlib.pyplot as plt # for plotting data
import shutil

In [ ]:
def create_qBOLD_masks(sub, working_dir, conds, GMthresh, T2thresh, R2thresh, CBVthresh = 20, T2Sthresh = 150, OEFthresh = 1, CBFthresh = 150):
    '''
    creates different qBOLD masks with appropriate thresholds
    GM, T2 and R2strich thresholds are obligatory, upper CBV, T2S, OEF and CBF defaults are given
    adjust function: path and labels are set qBOLD-specific
    Returns: masks: list with mask names and mask paths
    ''' 
    
    ## GM ## 
    GMseg = os.path.join(working_dir, 'qmri',  sub + '_space-T2_GM')
  
    #threshold and binarize GMseg file
    get_ipython().system(' ! fslmaths {GMseg}.nii -thr {GMthresh} -bin {GMseg}_mask.nii #mask with GM threshold   ')
    
    #if there's a mask with .nii and nii.gz, remove the .nii mask, otherwise it will give an multiple pictures error later on!
    if os.path.isfile(GMseg + '_mask.nii.gz') and os.path.isfile(GMseg + '_mask.nii'):
        os.remove(GMseg + '_mask.nii')    
    
    # copy file to main subject directory
    #shutil.move((GMseg + '_mask.nii.gz'), qBOLD_locfolder + '/GM_mask.nii.gz')
    
    ## T2thresh ## 
    T2 = os.path.join(working_dir, 'qmri',  sub + '_space-T2_T2map')
    #threshold and binarize T2 file
    get_ipython().system(' ! fslmaths {T2}.nii -uthr {T2thresh} -bin {T2}_mask.nii.gz #uthr: kick out values above thr')

    # copy file to main subject directory
   # shutil.move(os.path.join(working_dir, 'qmri', sub + 'space-T2_T2map_mask.nii.gz')
    
    ## R2strich ##
    R2prime_list = {}
    for k in range(len(conds)): #create mask for every condition
        R2prime = os.path.join(working_dir, 'qmri',  sub + '_task-'+ conds[k] +'_space-T2_R2prime')
        get_ipython().system(' ! fslmaths {R2prime}.nii -uthr {R2thresh} -bin {R2prime}_mask.nii.gz #uthr: kick out values above thr    ')
        R2prime_list[k] = R2prime + '_mask.nii.gz'
    R2prime_mask = os.path.join(working_dir, 'qmri',  sub + '_space-T2_R2prime_mask.nii.gz')  #create mask combining all cond masks

    if len(conds)==1:
        get_ipython().system(' ! fslmaths {R2prime_list[0]} -mul {R2prime_list[0]} {R2prime_mask} ')
    if len(conds)==2:
        get_ipython().system(' ! fslmaths {R2prime_list[0]} -mul {R2prime_list[1]} {R2prime_mask} ')
    if len(conds)==3:
        get_ipython().system(' ! fslmaths {R2prime_list[0]} -mul {R2prime_list[1]} -mul {R2prime_list[2]} {R2prime_mask} ')
    if len(conds)==4:
        get_ipython().system(' ! fslmaths {R2prime_list[0]} -mul {R2prime_list[1]} -mul {R2prime_list[2]} -mul {R2prime_list[3]} {R2prime_mask} ')
    if len(conds)>4:
        print('you have more than 4 conditions, not supported by this script')
    
                
    ## CBV 
    CBV =  os.path.join(working_dir, 'qmri',  sub + '_task-control_space-T2_cbv')
    #threshold and binarize T2 file
    get_ipython().system(' ! fslmaths {CBV}.nii -uthr {CBVthresh} -bin {CBV}_mask.nii.gz #uthr: kick out values above thr')
    
    # copy file to main subject directory
    #shutil.move((CBV + '_mask.nii.gz'), qBOLD_locfolder + '/CBV_control_mask.nii.gz')
    
    ## T2S ##
    T2S_list = {}
    for k in range(len(conds)): #create mask for every condition
        T2S =os.path.join(working_dir, 'qmri',  sub + '_task-'+ conds[k] +'_space-T2_T2Smap') 
        get_ipython().system(' ! fslmaths {T2S}.nii -uthr {T2Sthresh} -bin {T2S}_mask.nii.gz #uthr: kick out values above thr    ')
        T2S_list[k] = T2S + '_mask.nii.gz'
    T2S_mask = os.path.join(working_dir, 'qmri',  sub + '_space-T2_T2Smap_mask.nii.gz')  #create mask combining all cond masks

    if len(conds)==1:
        get_ipython().system(' ! fslmaths {T2S_list[0]} -mul {T2S_list[0]} {T2S_mask} ')
    if len(conds)==2:
        get_ipython().system(' ! fslmaths {T2S_list[0]} -mul {T2S_list[1]} {T2S_mask} ')
    if len(conds)==3:
        get_ipython().system(' ! fslmaths {T2S_list[0]} -mul {T2S_list[1]} -mul {T2S_list[2]} {T2S_mask} ')
    if len(conds)==4:
        get_ipython().system(' ! fslmaths {T2S_list[0]} -mul {T2S_list[1]} -mul {T2S_list[2]} -mul {T2S_list[3]} {T2S_mask} ')
    if len(conds)>4:
        print('you have more than 4 conditions, not supported by this script')

    
    ## OEF ##
    OEF_list = {}
    for k in range(len(conds)): #create mask for every condition
        OEF = os.path.join(working_dir, 'qmri',  sub + '_task-'+ conds[k] +'_space-T2_oef')
        get_ipython().system(' ! fslmaths {OEF}.nii -uthr {OEFthresh} -bin {OEF}_mask.nii.gz #uthr: kick out values above thr    ')
        OEF_list[k] = OEF + '_mask.nii.gz'
    OEF_mask =  os.path.join(working_dir, 'qmri',  sub + '_space-T2_oef_mask.nii.gz') #create mask combining all cond masks
  
    if len(conds)==1:
        get_ipython().system(' ! fslmaths {OEF_list[0]} -mul {OEF_list[0]} {OEF_mask} ')
    if len(conds)==2:
        get_ipython().system(' ! fslmaths {OEF_list[0]} -mul {OEF_list[1]} {OEF_mask} ')
    if len(conds)==3:
        get_ipython().system(' !fslmaths {OEF_list[0]} -mul {OEF_list[1]} -mul {OEF_list[2]} {OEF_mask} ')
    if len(conds)==4:
        get_ipython().system(' ! fslmaths {OEF_list[0]} -mul {OEF_list[1]} -mul {OEF_list[2]} -mul {OEF_list[3]} {OEF_mask} ')
    if len(conds)>4:
        print('you have more than 4 conditions, not supported by this script')


    ## CBF ##
    CBF_list = {}
    for k in range(len(conds)): #create mask for every condition
        CBF = os.path.join(working_dir, 'qmri',  sub + '_task-'+ conds[k] +'_space-T2_cbf')
        get_ipython().system(' ! fslmaths {CBF}.nii -uthr {CBFthresh} -bin {CBF}_mask.nii.gz #uthr: kick out values above thr    ')
        CBF_list[k] = CBF + '_mask.nii.gz'
    CBF_mask =  os.path.join(working_dir, 'qmri',  sub + '_space-T2_cbf_mask.nii.gz') #create mask combining all cond masks

    if len(conds)==1:
        get_ipython().system(' ! fslmaths {CBF_list[0]} -mul {CBF_list[0]} {CBF_mask} ')
    if len(conds)==2:
        get_ipython().system(' ! fslmaths {CBF_list[0]} -mul {CBF_list[1]} {CBF_mask} ')
    if len(conds)==3:
        get_ipython().system(' ! fslmaths {CBF_list[0]} -mul {CBF_list[1]} -mul {CBF_list[2]} {CBF_mask} ')
    if len(conds)==4:
        get_ipython().system(' ! fslmaths {CBF_list[0]} -mul {CBF_list[1]} -mul {CBF_list[2]} -mul {CBF_list[3]} {CBF_mask} ')
    if len(conds)>4:
        print('you have more than 4 conditions, not supported by this script')
        
    mask_list = [ os.path.join(working_dir, 'qmri',  sub + '_space-T2_GM_mask.nii.gz'), 
             os.path.join(working_dir, 'qmri',  sub + '_space-T2_T2map_mask.nii.gz'), 
             os.path.join(working_dir, 'qmri',  sub + '_space-T2_R2prime_mask.nii.gz'),
            os.path.join(working_dir, 'qmri', sub + '_task-control_space-T2_cbv.nii.gz'),
             os.path.join(working_dir, 'qmri',  sub + '_space-T2_T2Smap_mask.nii.gz'), 
             os.path.join(working_dir, 'qmri',  sub + '_space-T2_oef_mask.nii.gz'), 
              os.path.join(working_dir, 'qmri',  sub + '_space-T2_cbf_mask.nii.gz')]

    
    return mask_list

In [ ]:
def mask_nii(nifti, mask_list, out_name):
    '''
    This function takes a list of binary masks (list with string pairs: name, mask_filename) as inputs: 
    Example: mask_list = [('GM', GM_mask), 
             ('T2', T2_mask), 
             ('R2strich', R2strich_mask)} 
    If there is only one mask: it masks nifti with the mask
    If there are several masks: It multiplies all masks and then masks nifti with mask
    Returns: out_name
    '''
    if len(mask_list) == 0: 
        raise Exception('mask_list is empty!')
    
        
    for i, mask in enumerate(mask_list):
        print('Apply mask: ' + str(i))
        if i == 0:  
            get_ipython().system(' ! fslmaths {nifti} -mas {mask} {out_name}')
        else: 
            get_ipython().system(' ! fslmaths {out_name} -mas {mask} {out_name}')
            
    return out_name

In [146]:
def to1D(listOf3DArray, listOfName, listOfCond):
    """
    This function takes 3D maps of different parameters and converts the data into a pandas dataframe for further calculation

    Input:
        listOf3DArray: A list of 3D maps of relevant data stored in numpy 3D arrays

        listOfName: A list of the name of the parameters of the 3D maps. Accepted values are: "CBF","OEF","R2p","BOLD","CMRO2","lobe", "nw", "CBV". 
                    If other names are added, a warning but no error will be given. 
                    The script cannot use columns with other names, but it will keep them in the dataframe for your own interest.

        listOfCond: Condition of the 3D maps. For comparative maps such as BOLD, please use the concatenated string of treatment and control conditions. 
                    e.g.: if you are comparing "Calc" condition with "Rest" condition, please put "CalcRest" the BOLD map. The script is case-sensitive. 
                    Capitalisation of the first letter is recommended but not required.

        All three list should have the same length, and the arrays in listOf3DArray should have the same dimention

    Output:
        a dataframe, a tuple, and a 3D array

        Dataframe: A pandas dataframe with each voxel as a row and each parameter in the listOf3DArray as a column. This df will be used for downstream calculation.

        Tuple: Dimension of the 3D arrays. This allows you to double check the input. The output can also be used as the dimension input in the to3D() function.

        3D array: An index map mapping 3D index to row index in the dataframe. This can be used as the indexMap input in the getRange() function.
    """
    
    if not (len(listOf3DArray) == len(listOfName) == len(listOfCond)):
        raise Exception("lists of array, name, condition have different dimensions")
    
    output = pd.DataFrame()
    
    arr0 = listOf3DArray[0]
    x = arr0.shape[0]; y = arr0.shape[1]; z = arr0.shape[2]
    dimension = (x,y,z)
    
    for i in range(len(listOfName)):
        name = listOfName[i]
        arr = listOf3DArray[i]
        cond = listOfCond[i]
        
        # check the dimension of the input array
        if arr.shape[0] != x:
            raise Exception(str(i+1)+"th array has wrong x dimension")
        if arr.shape[1] != y:
            raise Exception(str(i+1)+"th array has wrong y dimension")
        if arr.shape[2] != z:
            raise Exception(str(i+1)+"th array has wrong z dimension")            
        # check the name of the input name
        if name not in ["CBF","OEF","R2p","BOLD","CMRO2","lobe", "nw", "CBV"]:
            warnings.warn(name + " is not in the default name list \n"+
                          "Default names are : CBF, OEF, R2p, BOLD, CMRO2, lobe, nw, CBV")
        
        output[name+cond] = arr.flatten()
    
    output.dropna(inplace = True)
    
    numEntity = x*y*z
    indexMap = np.arange(numEntity).reshape(x,y,z)
    
    return output, dimension, indexMap

In [147]:
def AverageFileReadandFilter(fileName, cond, IDs = None, thres = 0, rename = True):
    """
    This function reads in a csv file with Glasser median values and following column names: 
        'ID', 'vox_count': numer of voxels per parcel, 'cond': condition name, 'par':parameter, e.g. CMRO2, 'parcel': Glasser parcel number, 
        'nw': network, i.e. primary_visual, 'lobe': e.g. Occ, 'median': parcel median
    then filters the data according to the participant ID, conditions and vox counts, 
    and rearranges the data into a pandas dataframe that is suitable for downstream analysis.

    Input:
        fileName: Directory of the qBOLD output csv file

        cond: A list of two conditions of interest. e.g. ["calc", "control"]. Please make sure that the conditions match the cond entries of the csv file

        IDs: A list of integers representing the ID of participants of interest. Default is None. If default is used, all participants will be included.

        thres: The minimum number of voxel count for a parcel to be included in the analysis. Default is 0.

        rename: Rename the columns for downstream analysis. This will only work for the output of Samira's qBOLD script. 
                Default is true. Renaming the columns is necessary for downstream analysis. 
                If you set rename = False, please rename the column names yourself to variable name ("CBF, "CBV", etc) + condition with no space or underscore in between.

    Output:
        A pandas dataframe with each subject's parcel as row, and each distinct entity in the par column of the original csv as a column. 
        The rows with incomplete data after filtering will be removed.

    """
    
    df = pd.read_csv(fileName, index_col=0)
    if IDs:
        df = df.loc[df['ID'].isin(IDs)]
    if thres:
        df = df.loc[df['vox_count'] >= thres]
    df = df.loc[(df['cond'].isin(cond)) | (df['cond'] == cond[0]+cond[1])]
    df["par_cond"] = df["par"]+df["cond"]
    df = df.pivot_table(index=["ID","parcel","nw","lobe"], columns=["par_cond"],values="median")
    df.dropna(inplace = True, subset=['BOLD'+cond[0]+cond[1]])
    if rename:
        df.rename({'xCBF_3_BRmsk_CSF'+cond[0]: 'CBF'+cond[0],
                    'xCBF_3_BRmsk_CSF'+cond[1]: 'CBF'+cond[1],
                    'sR2strich'+cond[0]: 'R2p'+cond[0],
                    'sR2strich'+cond[1]: 'R2p'+cond[1],
                    'rOEF'+cond[0]: 'OEF'+cond[0],
                    'rOEF'+cond[1]: 'OEF'+cond[1]}, axis=1, inplace=True)
    df.reset_index(inplace=True)
    return df

In [106]:
def to3D(df, listOfColumnName, dimension):
    """
    This function generates 3D maps for calculated parameters.

    Input:
        df: The pandas dataframe storing the calculated values

        listOfColumnName: A list of column names for the parameters that 3D maps will be generated.

        dimension: A tuple of length 3 specifying the dimension of the 3D map. The second output of to1D() function can provide this tuple.

    Output:
        A dictionary with keys being the column names and values being the corresponding 3D maps.
    """
    
    DictOfOutput = {}
    for col in listOfColumnName:
        DictOfOutput[col] = np.array(df[col]).reshape(dimension)
    return DictOfOutput

In [107]:
def getRange(listOf3DCoord, indexMap):
    """
    This function takes a list of 3D indices and converts them to 1D row indices for the dataframe. This is used to perform a calculation on a specific set of voxels.

    Input:
        listOf3DCood: A list of tuples representing the 3D indices of the voxels of interest

        indexMap: A pre-generated 3D array map. The third output of to1D can provide this index map

    Output:
        A list of row numbers in the dataframe corresponding to the voxels of interest. This can be used as the applyRange input for applyFuncToDf().
    """
    
    listOf1DCoord = []
    for coord in listOf3DCoord:
        listOf1DCoord.append(indexMap[coord[0]][coord[1]][coord[2]])
    return listOf1DCoord

In [108]:
def getRegion(df, listOfRegion, regionType):
    """
    This function gives the row indices corresponding to a specific brain region.

    Input:
        df: The pandas dataframe storing the data

        region: A string specifying the region of interest, e.g. "Occ". The region must be an entity in the column of regionType.

        regionType: Column name of the specified region, e.g. "nw" or "lobe" or "parcel".
    Output:
        A list of row numbers in the dataframe corresponding to the region of interest. This can be used as the applyRange input for applyFuncToDf().
    """
    return list(df.loc[df[regionType].isin(listOfRegion)].index)

In [1]:
def getColName(func):
    """
    """
    colNameMap = {M:("M",[1]),
                 N:("N",[0,1]),
                 A:("a",[0]),
                 FickCMRO2:("FCMRO2",[0]),
                 FickCMRO2Ctrl:("FCMRO2",[1]),
                 FickCMRO2RelChange: ("rcFCMRO2",[0,1]),
                 DavisCMRO2RelChange: ("rcDCMRO2",[0,1]),
                 DavisCBFRelChange: ("rcDCBF",[0,1]),
                 DavisBOLD: ("DBOLD",[0,1]),
                 FujitaR2pRelChange: ("rcFR2P",[0,1]),
                 FujitaCMRO2RelChange: ("rcFCMRO2",[0,1])}
    if func in colNameMap:
        return colNameMap[func]
    else:
        raise Exception("No name is available for the operation, please use the colName parameter to define the output column name.")   

In [110]:
def preparation(cond, TE):
    """
    This function sets up the enviroment for downstream calculations. You should run it before you run each condition pair.

    Input:
        cond: A list of two conditions of interest. e.g. ["calc", "control"]. 
            Please make sure that the conditions is consistent with the ones in column names and that the first condition is the task condition and the second is the baseline.

        TE: echo time of EPI, in ms.

    Output:
        None
        sets up global variables: condG (conditions), TEG (echo time)
    """
    
    global condG; global TEG; global aG; global bG; global MG
    condG = cond
    TEG = TE
    aG = bG = MG = None

In [111]:
def applyFuncToDf(func, df, applyRange = [], 
                  TE = None, a = None, b = None, m = None,
                 colName = None):
    """
    This function calculates a new parameter based on existent parameters.
    Example: applyFuncToDf(rc("CBF"), data, colName = "rcCBF") --> creates new column in dataframe 'data' with relative change of CBF

    Input:
        Required:
            func:

              M: calculating scaling factor M based on R2p of the baseline condition
              N: calculating CBF,CMRO2 coupling
              A: calculating alpha constant based on CBF, CBV
              FickCMRO2: calculating CMRO2 based on Fick's principle for the treatment condition
              FickCMRO2Ctrl: calculating CMRO2 based on Fick's principle for the baseline condition
              FickCMRO2RelChange:calculating CMRO2 relative change based on Fick's principle
              DavisCMRO2RelChange: calculating CMRO2 relative change based on Davis model
              DavisCBFRelChange: calculating CBF relative change based on Davis model
              DavisBOLD: calculating BOLD signal based on Davis model
              FujitaR2pRelChange: Delta R2' calculated using BOLD data divided by baseline measured R2'
              FujitaCMRO2RelChange: Relative change of CMRO2 calculated using Fick's principle with relative CBF, CBV, and R2' data. Relative R2' data is calculated form FujitaR2pRelChange
              rc(Variable): A string argument is required to specify the variable for relative change, e.g. rc("R2p"). The colName parameter is required.

            df: The dataframe of the data
        
        Optional:
            TE: This allows you to change the echo time permanently on top of preparation.
            a: Alpha value, if not specified, alpha value will be extracted from the a column or calculated from CBF,CBV
            b: Beta value. Required for calculations using Davis Model.
            m: M value. if not specified, alpha value will be extracted from the M column or calculated from TE, R2p.
            colName: Column name of the newly calculated entity. Required for self-defined functions.

    Output:
        None. A new column will be added to df.
    """
    
    if not colName:
        funName, condList = getColName(func)
        colName = funName
        for i in condList:
            colName += condG[i]
    global TEG; global aG; global bG; global MG
    if TE:
        TEG = TE
        print("TE is changed")
    aG = a; bG = b; MG = m
    if a:
        colName += "_a"+str(a)
    if b:
        colName += "_b"+str(b)
    if m:
        colName += "_M"+str(M)
    if not applyRange:
        df[colName] = df.apply(func,axis=1)
    else:
        df.at[applyRange, colName] = (df.loc[set(applyRange)]).apply(func,axis=1)
    aG = bG = MG = None

In [141]:
def M(row):
    """
    calculates scaling factor M based on R2p of the baseline condition
    """
    if not TEG:
        raise Exception("Trying to calculate M but TE is not specified, please add argument TE = ?")
    return row["R2p"+condG[1]]*TEG/10

In [113]:
def N(row):
    """
    calculates delta CBF/ deltaCMRO2 coupling
    """
    dCBF = row["CBF"+condG[0]] - row["CBF"+condG[1]]
    dCMRO2 = row["CMRO2"+condG[0]] - row["CMRO2"+condG[1]]
    return dCBF/dCMRO2

In [5]:
def A(row):
    """
    calculates alpha constant: power-law relationship between CBV and CBF (CBV = 0.8 * CBF^a), solved for a
    """
    return math.log(row["CBV"+condG[1]]/0.8)/math.log(row["CBF"+condG[1]])

In [115]:
def FickCMRO2task(row):
    """
    calculates CMRO2 based on Fick's principle
    """
    return row["CaO2"+condG[0]]*row["OEF"+condG[0]]*row["CBF"+condG[0]]

In [117]:
def FickCMRO2RelChange(row):
    """
    calculates CMRO2 relative change based on Fick's principle
    """
    return row["OEF"+condG[0]]*row["CBF"+condG[0]]/(row["OEF"+condG[1]]*row["CBF"+condG[1]])-1

In [6]:
def DavisCheck(row):
    """
    """
    a = aG; b = bG; m = MG
    if (not a) and ("a"+condG[1] not in row.index):
        warnings.warn(str("alpha is not specified and a"+condG[1] + " is not in the dataframe\n"+
                      "Calculating alpha based on CBF and CBV now"))
        a = A(row)
    elif not a:
        a = row["a"+condG[1]]
    if (not b):
        raise Exception("beta is not specified")
    if (not m) and ("M"+condG[1] not in row.index):
        warnings.warn(str("M is not specified and M"+condG[1] + " is not in the dataframe\n"+
                      "Calculating M based on R2p and TE now"))
        m = M(row)
    elif not m:
        m = row["M"+condG[1]]
    return a, b, m

In [120]:
def DavisCMRO2RelChange(row):
    """
    calculates CMRO2 relative change based on Davis model
    """
    a, b, m = DavisCheck(row)
    product = 1 - row["BOLD"+condG[0]+condG[1]]/m
    rCMRO2 = (product/((row["CBF"+condG[0]]/row["CBF"+condG[1]])**(a-b)))**(1/b)
    return rCMRO2-1    

In [121]:
def DavisBOLD(row):
    """
    calculates BOLD signal based on Davis model
    """
    a, b, m = DavisCheck(row)
    return m*(1-(row["CBF"+condG[0]]/row["CBF"+condG[1]])**(a-b)*(row["CMRO2"+condG[0]]/row["CMRO2"+condG[1]])**b)-1

In [122]:
def DavisCBFRelChange(row):
    """
    calculates CBF relative change based on Davis model
    """
    a, b, m = DavisCheck(row)
    product = 1 - row["BOLD"+condG[0]+condG[1]]/m
    rCBF = (product/((row["CMRO2"+condG[0]]/row["CMRO2"+condG[1]])**(b)))**(1/(a-b))
    return rCBF-1    

In [1]:
def FujitaR2pRelChange(row):
    """
    Delta R2' calculated using BOLD data divided by baseline measured R2'
    """
    return -row["BOLD"+condG[0]+condG[1]]/TEG/row["R2p"+condG[1]]*10 ##*10 because TE is in msec
    #return -row["BOLD"+condG[0]+condG[1]]/TEG/row["R2p"+condG[1]]

##or the same as BOLD signal change?
#def FujitaR2pRelChange(row):
#    return row["BOLD"+condG[0]+condG[1]]

In [2]:
def FujitaR2ptask_nii(R2_baseline, BOLD_change, TE):
    """
    Input
        R2_baseline: R2' (as numpy 3D array) in baseline
        BOLD_change: relative BOLD signal change as numpy 3D array (task versus baseline, values between 0 and 1)
    Output
        R2_task: R2' in task as np 3D array, calculated via Fujita's approach
    """
    rel_change_R2 = -BOLD_change/TE/R2_baseline*10 ##*10 because TE is in msec
    
    R2_task = R2_baseline + (rel_change_R2 * R2_baseline)  
    R2_task[np.isnan(R2_task)]=0  # turn nans into zeros
    ## clear unreasonable high values: double the max values of R2' control
    max_val = np.max(R2_baseline)
    #R2_task[R2_task>(max_val*2)] = max_val*2
    
    return R2_task

In [3]:
def FujitaCMRO2RelChange(row):
    """
    Relative change of CMRO2 calculated using Fick's principle with relative CBF, CBV, and R2' data. Relative R2' data is calculated from FujitaR2pRelChange
    """
    ratioCBF = row["CBF"+condG[0]]/row["CBF"+condG[1]]
    ratioR2p = FujitaR2pRelChange(row)+1
    ratioCBV = row["CBV"+condG[0]]/row["CBV"+condG[1]]
    #return ratioCBF*ratioR2p*ratioCBV #this is the ratio CMRO2/CMRO2,0
    return ratioCBF*ratioR2p*ratioCBV-1 #why -1? because we want the relative change and not the ratio


In [ ]:
def FujitaOEFtask_nii(R2_baseline, R2_task, OEF_baseline, OEF_task):
    """
    Input
        R2_baseline: R2' (as numpy 3D array) in baseline
        R2_task: R2' (as numpy 3D array) in task
        OEF_baseline: OEF (as numpy 3D array) in baseline
        OEF_task: OEF (as numpy 3D array) in task
    Output
        OEF_task: R2' in task as np 3D array, calculated via Fujita's approach
    """
    rel_change_R2 = -BOLD_change/TE/R2_baseline*10 ##*10 because TE is in msec
    
    R2_task = R2_baseline + (rel_change_R2 * R2_baseline)  
    R2_task[np.isnan(R2_task)]=0  # turn nans into zeros
    ## clear unreasonable high values: double the max values of R2' control
    max_val = np.max(R2_baseline)
    R2_task[R2_task>(max_val*2)] = max_val*2
    
    return R2_task

In [ ]:
def DavisCMRO2task_nii(BOLD_percchange, CBF_percchange, R2strich_base, TEG, a=-0.05, b=0.98):
    """
    Input
        BOLD_percchange = 3D matrix with percent signal change values per voxel
        CBF_percchange = 3D matrix with percent signal change values per voxel
        a = -0.05, b=0.98 ## see paper of Gagnon et al., 2016

    Output
        CMRO2 in percent singal change, 3D matrix, one value per voxel
    """

# Davis model
#Z = M*(1-((Y+1)**(a-b))*(X+1)**b)
# X+1 = ((1-(Z/M))**(1/b)*(Y+1)**(1-(a/b))) ## resolved for rel. change in CMRO2 (X+1 = CMRO2task/CMRO2baseline; X = percchange CMRO2, +1 turns it into relative change)

    M = R2strich_base*TEG/10
    CMRO2 = (1-(BOLD_percchange/M))**(1/b)*(CBF_percchange+1)**(1-(a/b))
    
    return (CMRO2-1)

In [ ]:
def rc(var):
    """
    A string argument is required to specify the variable for relative change, e.g. rc("R2p"). The colName parameter is required.
    Example: applyFuncToDf(rc("CBF"), data, colName = "rcCBF") --> creates new column in dataframe 'data' with relative change of CBF
    """
    return lambda row: (row[var+condG[0]] - row[var+condG[1]])/row[var+condG[1]]